**Author**: Andy Tzanidakis \
Last updated: May 05, 2024

## Overview



### VizieR Query


### Crossmatch to ZTF

### Compute Time-Series Featues with `TAPE`

### Exercise

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams
rcParams['savefig.dpi'] = 550
rcParams['font.size'] = 20
plt.rc('font', family='serif')
import matplotlib as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams['axes.linewidth'] = 2

import lsdb
import tape

import dask
dask.config.set({"temporary-directory" :'/epyc/ssd/users/atzanida/tmp'})
dask.config.set({"dataframe.shuffle-compression": 'Snappy'})

from dask.distributed import Client

/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/dask/config.py:779: FutureWarning: Dask configuration key 'dataframe.shuffle-compression' has been deprecated; please use 'dataframe.shuffle.compression' instead
  warnings.warn(


## VizieR Querying

In [2]:
## ViZier and Aladin querying 
from pyvo import registry  # version >=1.4.1 
from mocpy import MOC
from ipyaladin import Aladin

In [3]:
# the catalogue name in VizieR (Gaia DR3 part 6)
CATALOGUE = "I/360"

catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"

# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

tables = voresource.get_tables()

# We can also extract the tables names for later use
tables_names = list(tables.keys())

In [4]:
print (f"Available table names: {tables_names}")

Available table names: ['I/360/syntphot', 'I/360/goldu', 'I/360/goldsp', 'I/360/goldsa', 'I/360/goldoba', 'I/360/goldf', 'I/360/goldc', 'I/360/binmass']


In [5]:
# Let's read quickly the table description...
voresource.describe(verbose=True)

Gaia DR3 Part 6. Performance verification
Short Name: I/360
IVOA Identifier: ivo://cds.vizier/i/360
Access modes: conesearch, hips#hips-1.0, tap#aux, web
Multi-capabilty service -- use get_service()

Gaia Data Release 3 (Gaia DR3) will be released on 13 June 2022. The Gaia DR3
catalogue builds upon the Early Data Release 3 (released on 3 December 2020)
and combines, for the same stretch of time and the same set of observations,
these already-published data products with numerous new data products such as
extended objects and non-single stars.

Subjects: I/360
Waveband Coverage: optical
More info: https://cdsarc.cds.unistra.fr/viz-bin/cat/I/360


In [8]:
# Select the first table name
table_name_1 = "I/360/goldf"
first_table_name = table_name_1

In [57]:
tap_service = voresource.get_service("tap")
tap_records = voresource.get_service("tap").run_sync(f'SELECT TOP 2000000  * \
                            FROM "{first_table_name}" WHERE (DE_ICRS > -30)')

In [58]:
table0 = tap_records.to_table()

# Convert to pandas dataframe
table_df = table0.to_pandas()

In [59]:
# select only M-type stars
table_df = table_df[table_df['SpType'].str.contains('M')]

In [62]:
table_df.head(1)

,recno,Source,Teff-P,logg-P,[M/H]-P,AG-P,E(BP-RP)-P,[alphe/Fe]-S,Teff-S,logg-S,...,Rad-FS,Lum-FS,Mass-FS,Age-FS,evol-FS,SpType,RA_ICRS,DE_ICRS,_RA_icrs,_DE_icrs
188,1015607,2906146503664238592,3699.9863,4.7953,-0.2256,0.3829,0.2239,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<NA>,M,80.970933,-29.992279,80.970933,-29.992279


In [34]:
client = Client(n_workers=4, threads_per_worker=1, memory_limit='auto')

/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45537 instead
  warnings.warn(


In [35]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:45537/status,
Dashboard: http://127.0.0.1:45537/status,Workers: 4
Total threads: 4,Total memory: 41.98 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36703,Workers: 4
Dashboard: http://127.0.0.1:45537/status,Total threads: 4
Started: Just now,Total memory: 41.98 GiB
Comm: tcp://127.0.0.1:34512,Total threads: 1
Dashboard: http://127.0.0.1:42241/status,Memory: 10.50 GiB
Nanny: tcp://127.0.0.1:46499,


In [63]:
%%time
hips_object = lsdb.from_dataframe(
    table_df,
    catalog_name="golden",
    catalog_type="object",
    ra_column="RA_ICRS", 
    dec_column="DE_ICRS")

In [37]:
hips_object

,recno,Source,Teff-P,logg-P,[M/H]-P,AG-P,E(BP-RP)-P,[alphe/Fe]-S,Teff-S,logg-S,[M/H]-S,Rad-F,Lum-F,Mass-F,Age-F,evol-F,Rad-FS,Lum-FS,Mass-FS,Age-FS,evol-FS,SpType,RA_ICRS,DE_ICRS,_RA_icrs,_DE_icrs,Norder,Dir,Npix
npartitions=27,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,int64,int64,float64,float64,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,Int16,float64,float64,float64,float64,Int16,string,float64,float64,float64,float64,uint8,uint64,uint64
1152921504606846976,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12682136550675316736,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18446744073709551615,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [38]:
hips_object_mtype = hips_object.query('SpType=="M"')

In [40]:
%%time
hips_object_mtype.head(1)

CPU times: user 116 ms, sys: 47.1 ms, total: 163 ms
Wall time: 497 ms


,recno,Source,Teff-P,logg-P,[M/H]-P,AG-P,E(BP-RP)-P,[alphe/Fe]-S,Teff-S,logg-S,...,Age-FS,evol-FS,SpType,RA_ICRS,DE_ICRS,_RA_icrs,_DE_icrs,Norder,Dir,Npix
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
197754504609792,3267192,98857263018112,3542.0205,4.8119,-0.6898,0.4936,0.2947,NaN,NaN,NaN,...,NaN,<NA>,M,44.655283,0.729574,44.655283,0.729574,0,0,0


## Load Additional Catalogs

In [48]:
# load ZTF source table
ztf_sources = lsdb.read_hipscat("/epyc/data3/hipscat/catalogs/ztf_axs/ztf_zource")

# load ZTF object table
ztf = lsdb.read_hipscat("/epyc/data3/hipscat/catalogs/ztf_axs/ztf_dr14",
                        columns=['ps1_objid', 'nobs_r', 'nobs_g', 'ra', 'dec']) # select only Nobs (gr) bands

In [49]:
ztf

,ps1_objid,nobs_r,nobs_g,ra,dec
npartitions=2352,,,,,
0,int64,int32,int32,float64,float64
18014398509481984,...,...,...,...,...
...,...,...,...,...,...
13830554455654793216,...,...,...,...,...
18446744073709551615,...,...,...,...,...


## Crossmatch

In [50]:
xmatch_golden_ztf_object = hips_object_mtype.crossmatch(ztf, n_neighbors=1, radius_arcsec=1, require_right_margin=False)

/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/lsdb/dask/crossmatch_catalog_data.py:117: RuntimeWarning: Right catalog does not have a margin cache. Results may be inaccurate
  warnings.warn("Right catalog does not have a margin cache. Results may be inaccurate", RuntimeWarning)


In [52]:
xmatch_golden_ztf_object.head(1)

,recno_golden,Source_golden,Teff-P_golden,logg-P_golden,[M/H]-P_golden,AG-P_golden,E(BP-RP)-P_golden,[alphe/Fe]-S_golden,Teff-S_golden,logg-S_golden,...,_DE_icrs_golden,Norder_golden,Dir_golden,Npix_golden,ps1_objid_ztf_dr14,nobs_r_ztf_dr14,nobs_g_ztf_dr14,ra_ztf_dr14,dec_ztf_dr14,_dist_arcsec
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
197754504609792,3267192,98857263018112,3542.0205,4.8119,-0.6898,0.4936,0.2947,NaN,NaN,NaN,...,0.729574,0,0,0,108870446552145874,386,373,44.655283,0.729621,0.16896


In [54]:
%%time

# Head of xmatch table with 250k rows for exploration
xmatch_golden_ztf_object_demo = xmatch_golden_ztf_object.head(250_000)

KeyboardInterrupt: 

In [53]:
%%time
# sync source catalog data to object (xmatched) object
_sources = xmatch_golden_ztf_object.join(
    ztf_sources, left_on="ps1_objid_ztf_dr14", right_on="ps1_objid")

/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/lsdb/dask/join_catalog_data.py:195: RuntimeWarning: Right catalog does not have a margin cache. Results may be inaccurate
  warnings.warn("Right catalog does not have a margin cache. Results may be inaccurate", RuntimeWarning)


CPU times: user 15.9 s, sys: 1.01 s, total: 16.9 s
Wall time: 16.2 s
